In [1]:
import numpy as np
import scipy as sp
import scipy.signal as sig
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, regularizers, callbacks
from data_wrangling.datamanager import DataLoader
import pickle
import sys

In [2]:
dl = DataLoader()
data = dl.get_fcx2(['s1'])

In [3]:
X1 = data['s1']['data']
mn = X1.mean(0).reshape((1, -1))
st = X1.std (0).reshape((1, -1))
X1 = (X1 - mn) / st
X1.shape

(1303952, 73)

In [5]:
fs = 1000

idxs_train_test_split = int(len(X1) * 0.7)

p_in  = 100
p_out = 10

step  = 15

idxs_train = np.arange(0, idxs_train_test_split - p_out - p_in, step)

X = np.stack([
    X1[idx : idx + p_in]
    for idx in idxs_train
])

Y = np.stack([
    X1[idx + p_in : idx + p_in + p_out]
    for idx in idxs_train
])

X.shape, Y.shape

((60844, 100, 73), (60844, 10, 73))

In [9]:
model = keras.Sequential([
    layers.Input(X.shape[1:]),
    layers.Bidirectional(
        layers.LSTM(
            64, 
            return_sequences = True
        )
    ),
    layers.Lambda(lambda inputs: inputs[:, -p_out:, :]),
    layers.Bidirectional(
        layers.LSTM(
            64, 
            return_sequences = True
        )
    ),
    layers.Dense(Y.shape[-1])
])

model.compile(loss = 'mse', optimizer = 'adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 100, 128)          70656     
_________________________________________________________________
lambda (Lambda)              (None, 10, 128)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 10, 128)           98816     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 73)            9417      
Total params: 178,889
Trainable params: 178,889
Non-trainable params: 0
_________________________________________________________________


In [10]:
hist = model.fit(
    X,
    Y,
    epochs = 50,
    batch_size = 2048,
    validation_split = 0.2,
    callbacks = [callbacks.EarlyStopping(min_delta = 1e-3, patience = 2)]
)

Epoch 1/50
24/24 [==============================] - 8s 167ms/step - loss: 0.8719 - val_loss: 0.5511
Epoch 2/50
24/24 [==============================] - 3s 120ms/step - loss: 0.4426 - val_loss: 0.3668
Epoch 3/50
24/24 [==============================] - 3s 120ms/step - loss: 0.3041 - val_loss: 0.2841
Epoch 4/50
24/24 [==============================] - 3s 123ms/step - loss: 0.2417 - val_loss: 0.2390
Epoch 5/50
24/24 [==============================] - 3s 120ms/step - loss: 0.2058 - val_loss: 0.2098
Epoch 6/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1812 - val_loss: 0.1914
Epoch 7/50
24/24 [==============================] - 3s 118ms/step - loss: 0.1644 - val_loss: 0.1753
Epoch 8/50
24/24 [==============================] - 3s 119ms/step - loss: 0.1515 - val_loss: 0.1657
Epoch 9/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1428 - val_loss: 0.1586
Epoch 10/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1343 - val_loss: 0.1491

In [12]:
hist = model.fit(
    X,
    Y,
    epochs = 50,
    batch_size = 128,
    validation_split = 0.3,
    callbacks = [callbacks.EarlyStopping(min_delta = 5e-4, patience = 4)]
)

Epoch 1/50
333/333 [==============================] - 7s 21ms/step - loss: 0.0808 - val_loss: 0.0840
Epoch 2/50
333/333 [==============================] - 7s 20ms/step - loss: 0.0720 - val_loss: 0.0785
Epoch 3/50
333/333 [==============================] - 7s 20ms/step - loss: 0.0674 - val_loss: 0.0737
Epoch 4/50
333/333 [==============================] - 6s 19ms/step - loss: 0.0641 - val_loss: 0.0717
Epoch 5/50
333/333 [==============================] - 6s 18ms/step - loss: 0.0611 - val_loss: 0.0685
Epoch 6/50
333/333 [==============================] - 6s 18ms/step - loss: 0.0594 - val_loss: 0.0672
Epoch 7/50
333/333 [==============================] - 6s 18ms/step - loss: 0.0573 - val_loss: 0.0671
Epoch 8/50
333/333 [==============================] - 6s 18ms/step - loss: 0.0573 - val_loss: 0.0722
Epoch 9/50
333/333 [==============================] - 6s 18ms/step - loss: 0.0557 - val_loss: 0.0643
Epoch 10/50
333/333 [==============================] - 6s 18ms/step - loss: 0.0538 - val_lo

In [13]:
pred = []
real = []
for idx in range(idxs_train_test_split, len(X1) - p_out - p_in, p_out):
    x = X1[idx : idx + p_in].reshape((1, p_in, -1))
    y = X1[idx + p_in : idx + p_in + p_out]
    pred.append(x)
    real.append(y)
    
per_stride = 2048
strides = 1 + len(pred) // per_stride
pred = np.concatenate([
    model(
        np.concatenate(pred[stride * per_stride : (stride + 1) * per_stride])
    ).numpy()
    for stride in range(strides)
])

real = np.stack(real)